# M5-Accuracy

## Inspiration

* https://www.kaggle.com/ajinomoto132/reduce-mem
* https://www.kaggle.com/zmnako/lgbm-update-0-85632
* https://www.kaggle.com/ratan123/m5-forecasting-lightgbm-with-timeseries-splits
* https://www.kaggle.com/ragnar123/very-fst-model

## Import Statements

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

%load_ext autoreload
%autoreload 2

INPUT = '/home/jupyter/'

## Helper functions

In [3]:
import pandas as pd
import numpy as np
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

# from: https://www.kaggle.com/ajinomoto132/reduce-mem
def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    """

    start_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
class M5Split2:
    
    def __init__(self, n_splits, group_id, date_col, train_size, gap_size, val_size, step):
        self.n_splits = n_splits + 1
        self.group_id = group_id
        self.date_col = date_col
        self.train_size = train_size
        self.gap_size = gap_size
        self.val_size = val_size
        self.step = step
        
        
    def split(self, df):
        df = df.sort_values(by=[self.date_col])
        indexes = []
        group_indexes = np.array(df.groupby(self.group_id, observed=True).apply(lambda x: x.index))
        for split in range(self.n_splits, 0, -1):
            val_idx = []
            gap_idx = []
            train_idx = []
                
            for idx_arr in group_indexes:
                
                if self.train_size + self.gap_size + self.val_size + self.step*split > len(idx_arr):
                    print(f'Max Split reached')
                    break
                    
                val_idx += list(idx_arr[-(self.val_size + self.step*(split-1)):len(idx_arr) - 1 - self.step*(split-1)])
                gap_idx += list(idx_arr[-(self.gap_size + self.val_size + self.step*(split-1)):-(self.val_size + self.step*(split-1))])
                train_idx += list(idx_arr[-(self.train_size + self.gap_size + self.val_size + self.step*(self.n_splits)):-(self.val_size + self.gap_size + self.step*(split-1))])
                
            yield train_idx, gap_idx, val_idx


In [5]:
# https://www.kaggle.com/ragnar123/very-fst-model
def simple_fe_extra(data):
    
    # rolling demand features
    data['lag_t56'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56))
    data['lag_t57'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(57))
    data['lag_t58'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(58))
    data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(7).mean())
    data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(7).std())
    data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(30).mean())
    data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(90).mean())
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(180).mean())
    data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(30).std())
    data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(30).skew())
    data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(30).kurt())
    
    
    # price features
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    # time features
    data['date'] = pd.to_datetime(data['date'])
    data['week'] = data['date'].dt.week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    
    
    return data


## Preprocessing

In [5]:
# ------------------------- CALENDAR ------------------------ #
df_calendar = pd.read_csv(filepath_or_buffer=f'{INPUT}calendar.csv')
df_calendar = reduce_mem_usage(df_calendar)
print(df_calendar.shape)
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
df_calendar['weekday'] = df_calendar['weekday'].astype(str)
df_calendar['d'] = df_calendar['d'].astype(str)
df_calendar['event_name_1'] = df_calendar['event_name_1'].astype(str)
df_calendar['event_type_1'] = df_calendar['event_type_1'].astype(str)
df_calendar['event_name_2'] = df_calendar['event_name_2'].astype(str)
df_calendar['event_type_2'] = df_calendar['event_type_2'].astype(str)

df_calendar['event_tomorrow_1'] = df_calendar['event_name_1'].shift(-1)
df_calendar['event_tomorrow_2'] = df_calendar['event_name_2'].shift(-1)
df_calendar['event_type_tomorrow_1'] = df_calendar['event_type_1'].shift(-1)
df_calendar['event_type_tomorrow_2'] = df_calendar['event_type_2'].shift(-1)

df_calendar = df_calendar.fillna(value='nan')

Memory usage of dataframe is 0.21 MB
Memory usage after optimization is: 0.22 MB
Decreased by -5.7%
(1969, 14)


In [6]:
# event_name_1 and event_name_2 should be fitted together since both are essentailly the same
le1 = LabelEncoder()
le1.fit(pd.concat(objs=[df_calendar['event_name_1'], df_calendar['event_name_2']], axis=0))
df_calendar['event_name_1'] = le1.transform(df_calendar['event_name_1'])
df_calendar['event_name_2'] = le1.transform(df_calendar['event_name_2'])

# event_type_1 and event_type_2 should be fitted together since both are essentailly the same
le2 = LabelEncoder()
le2.fit(pd.concat(objs=[df_calendar['event_type_1'], df_calendar['event_type_2']], axis=0))
df_calendar['event_type_1'] = le2.transform(df_calendar['event_type_1'])
df_calendar['event_type_2'] = le2.transform(df_calendar['event_type_2'])

le3 = LabelEncoder()
le3.fit(pd.concat(objs=[df_calendar['event_tomorrow_1'], df_calendar['event_tomorrow_2']], axis=0))
df_calendar['event_tomorrow_1'] = le3.transform(df_calendar['event_tomorrow_1'])
df_calendar['event_tomorrow_2'] = le3.transform(df_calendar['event_tomorrow_2'])

le4 = LabelEncoder()
le4.fit(pd.concat(objs=[df_calendar['event_type_tomorrow_1'], df_calendar['event_type_tomorrow_2']], axis=0))
df_calendar['event_type_tomorrow_1'] = le4.transform(df_calendar['event_type_tomorrow_1'])
df_calendar['event_type_tomorrow_2'] = le4.transform(df_calendar['event_type_tomorrow_2'])


df_calendar = reduce_mem_usage(df_calendar)

Memory usage of dataframe is 0.18 MB
Memory usage after optimization is: 0.15 MB
Decreased by 19.8%


In [7]:
df_calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_tomorrow_1,event_tomorrow_2,event_type_tomorrow_1,event_type_tomorrow_2
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,30,4,30,4,0,1,1,30,30,4,4
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,30,4,30,4,0,0,0,30,30,4,4
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,30,4,30,4,0,0,0,30,30,4,4
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,30,4,30,4,0,0,0,16,7,3,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,16,3,7,0,0,0,0,30,30,4,4


In [8]:
# ------------------------- SELLING PRICES ------------------------ #
df_selling_prices = pd.read_csv(f'{INPUT}sell_prices.csv')
print(df_selling_prices.shape)
df_selling_prices['store_id'] = df_selling_prices['store_id'].astype(str)
df_selling_prices['item_id'] = df_selling_prices['item_id'].astype(str)
df_selling_prices['sell_price_cents'] = df_selling_prices['sell_price'] - df_selling_prices['sell_price'].astype(int) 
df_selling_prices['sell_price_preceived'] = df_selling_prices['sell_price'].astype(int)
df_selling_prices = reduce_mem_usage(df_selling_prices)

(6841121, 4)
Memory usage of dataframe is 313.16 MB
Memory usage after optimization is: 91.44 MB
Decreased by 70.8%


In [9]:
def generate_cols(factor=0.5):
    cols =['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    start = int(1914*(1-factor))
    days = ['d_' + str(i) for i in range(start, 1914, 1)]
    cols += days
    return cols

columns = generate_cols(factor=0.5)

In [10]:
# ------------------------- SALES TRAIN VALIDATION ---------------------- #
df_sales_train_val = pd.read_csv(f'{INPUT}sales_train_validation.csv', usecols=columns)
df_sales_train_val['id'] = df_sales_train_val['id'].astype(str)
df_sales_train_val['item_id'] = df_sales_train_val['item_id'].astype(str)
df_sales_train_val['store_id'] = df_sales_train_val['store_id'].astype(str)
df_sales_train_val['dept_id'] = df_sales_train_val['dept_id'].astype(str)
df_sales_train_val['cat_id'] = df_sales_train_val['cat_id'].astype(str)
df_sales_train_val['state_id'] = df_sales_train_val['state_id'].astype(str)

print(f'Shape before melting: {df_sales_train_val.shape}')
print(f"Unique products before melting: {df_sales_train_val['id'].nunique()}")

df_sales_train_val = pd.melt(df_sales_train_val, id_vars=['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id'], var_name='d',
                              value_name='demand')

print(f'\nShape after melting: {df_sales_train_val.shape}')
print(f"Unique products after melting: {df_sales_train_val['id'].nunique()}")

# 0 means validation set
df_sales_train_val['set'] = 0
df_sales_train_val['strategy'] = 'train'
df_sales_train_val['id'] = df_sales_train_val['item_id'].str.cat(df_sales_train_val['store_id'], sep='_')


print(f'\nShape after making id from item_id and store_id: {df_sales_train_val.shape}')
print(f"Unique products after id from item_id and store_id: {df_sales_train_val['id'].nunique()}")

Shape before melting: (30490, 963)
Unique products before melting: 30490

Shape after melting: (29178930, 8)
Unique products after melting: 30490

Shape after making id from item_id and store_id: (29178930, 10)
Unique products after id from item_id and store_id: 30490


In [11]:
df_sales_train_val = df_sales_train_val.drop_duplicates()
print(f'Shape after making id from item_id and store_id: {df_sales_train_val.shape}')
print(f"Unique products after id from item_id and store_id: {df_sales_train_val['id'].nunique()}")

Shape after making id from item_id and store_id: (29178930, 10)
Unique products after id from item_id and store_id: 30490


In [12]:
df_sales_train_val = reduce_mem_usage(df_sales_train_val)

Memory usage of dataframe is 2448.79 MB
Memory usage after optimization is: 613.83 MB
Decreased by 74.9%


In [13]:
# --------------------- ADDING CALENDAR AND SELLING PRICES to SALES TRAIN VAL ---------------------- #
df_sales_train_val = df_sales_train_val.merge(df_calendar, on = ['d'], how='left')
df_sales_train_val = df_sales_train_val.merge(df_selling_prices, on = ["store_id", "item_id", "wm_yr_wk"], how='left')

In [14]:
df_sales_train_val.head()

,id,item_id,store_id,dept_id,cat_id,state_id,d,demand,set,strategy,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_tomorrow_1,event_tomorrow_2,event_type_tomorrow_1,event_type_tomorrow_2,sell_price,sell_price_cents,sell_price_preceived
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,CA_1,HOBBIES_1,HOBBIES,CA,d_957,0,0,train,2013-09-11,11333,Wednesday,5,9,2013,30,4,30,4,0,1,1,30,30,4,4,8.26,0.26,8.0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,CA_1,HOBBIES_1,HOBBIES,CA,d_957,0,0,train,2013-09-11,11333,Wednesday,5,9,2013,30,4,30,4,0,1,1,30,30,4,4,3.97,0.97,3.0
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,CA_1,HOBBIES_1,HOBBIES,CA,d_957,0,0,train,2013-09-11,11333,Wednesday,5,9,2013,30,4,30,4,0,1,1,30,30,4,4,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,CA_1,HOBBIES_1,HOBBIES,CA,d_957,1,0,train,2013-09-11,11333,Wednesday,5,9,2013,30,4,30,4,0,1,1,30,30,4,4,4.64,0.64,4.0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,CA_1,HOBBIES_1,HOBBIES,CA,d_957,1,0,train,2013-09-11,11333,Wednesday,5,9,2013,30,4,30,4,0,1,1,30,30,4,4,3.08,0.08,3.0


In [15]:
print(f'Shape after merging: {df_sales_train_val.shape}')

Shape after merging: (29178930, 30)


In [16]:
df_sales_train_val.shape

(29178930, 30)

In [17]:
df_sales_train_val = df_sales_train_val.reset_index(drop=True)

In [18]:
df_sales_train_val.columns

Index(['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id', 'd',
       'demand', 'set', 'strategy', 'date', 'wm_yr_wk', 'weekday', 'wday',
       'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'event_tomorrow_1',
       'event_tomorrow_2', 'event_type_tomorrow_1', 'event_type_tomorrow_2',
       'sell_price', 'sell_price_cents', 'sell_price_preceived'],
      dtype='object')

In [19]:
le = []
for num, col in enumerate(['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id']):
    le_id = LabelEncoder()
    le.append(le_id)
    le[num].fit(df_sales_train_val[col])
    df_sales_train_val[f'{col}_encoded'] = le[num].transform(df_sales_train_val[col])

In [20]:
df_sales_train_val.columns

Index(['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id', 'd',
       'demand', 'set', 'strategy', 'date', 'wm_yr_wk', 'weekday', 'wday',
       'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'event_tomorrow_1',
       'event_tomorrow_2', 'event_type_tomorrow_1', 'event_type_tomorrow_2',
       'sell_price', 'sell_price_cents', 'sell_price_preceived', 'id_encoded',
       'item_id_encoded', 'store_id_encoded', 'dept_id_encoded',
       'cat_id_encoded', 'state_id_encoded'],
      dtype='object')

In [21]:
# -------------------------- SAMPLE SUBMISSION -------------------------- #
df_sample_submission = pd.read_csv(f'{INPUT}sample_submission.csv')
df_sample_submission = reduce_mem_usage(df_sample_submission)
df_sample_submission['id'] = df_sample_submission['id'].astype(str)

Memory usage of dataframe is 13.49 MB
Memory usage after optimization is: 4.83 MB
Decreased by 64.2%


In [22]:
df_sample_submission.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
df_sample_submission['state_id'] = df_sample_submission['id'].str[-15:-13]
df_sample_submission['store_id'] = df_sample_submission['id'].str[-15:-11]
df_sample_submission['item_id'] = df_sample_submission['id'].str[0:-16]
df_sample_submission['dept_id'] = df_sample_submission['id'].str[0:-20]
df_sample_submission['cat_id'] = df_sample_submission['id'].str[0:-22]
df_sample_submission.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,state_id,store_id,item_id,dept_id,cat_id
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,WI,WI_3,FOODS_3_823,FOODS_3,FOODS
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,WI,WI_3,FOODS_3_824,FOODS_3,FOODS
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,WI,WI_3,FOODS_3_825,FOODS_3,FOODS
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,WI,WI_3,FOODS_3_826,FOODS_3,FOODS
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,WI,WI_3,FOODS_3_827,FOODS_3,FOODS


In [24]:
# -------------------------- SALES TEST -------------------------- #
df_sales_test = pd.melt(df_sample_submission, id_vars=['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id'], var_name='d', value_name='demand')

df_sales_test['d2'] = df_sales_test['d'].str[1:]
df_sales_test['d2'] = df_sales_test['d2'].astype(int)
df_sales_test.loc[df_sales_test['id'].str[-10:]=='evaluation', 'd2'] = df_sales_test.loc[df_sales_test['id'].str[-10:]=='evaluation', 'd2'] + 28
df_sales_test['d2'] = df_sales_test['d2'] + 1913
df_sales_test['d2'] = df_sales_test['d2'].astype(str)
df_sales_test['d2'] = 'd_' + df_sales_test['d2']

# 1 indicates test set
df_sales_test['set'] = 1
df_sales_test.loc[df_sales_test['id'].str[-10:]=='validation', 'strategy'] = 'validation'
df_sales_test.loc[df_sales_test['id'].str[-10:]=='evaluation', 'strategy'] = 'evaluation'
df_sales_test['original_id'] = df_sales_test['id']
df_sales_test['id'] = df_sales_test['id'].str[0:-11]

df_sales_test = df_sales_test.drop(columns=['d'])
df_sales_test = df_sales_test.rename(columns={'d2': 'd'})

In [25]:
df_sales_test.tail()

,id,item_id,store_id,dept_id,cat_id,state_id,demand,d,set,strategy,original_id
1707435,FOODS_3_823_WI_3,FOODS_3_823,WI_3,FOODS_3,FOODS,WI,0,d_1969,1,evaluation,FOODS_3_823_WI_3_evaluation
1707436,FOODS_3_824_WI_3,FOODS_3_824,WI_3,FOODS_3,FOODS,WI,0,d_1969,1,evaluation,FOODS_3_824_WI_3_evaluation
1707437,FOODS_3_825_WI_3,FOODS_3_825,WI_3,FOODS_3,FOODS,WI,0,d_1969,1,evaluation,FOODS_3_825_WI_3_evaluation
1707438,FOODS_3_826_WI_3,FOODS_3_826,WI_3,FOODS_3,FOODS,WI,0,d_1969,1,evaluation,FOODS_3_826_WI_3_evaluation
1707439,FOODS_3_827_WI_3,FOODS_3_827,WI_3,FOODS_3,FOODS,WI,0,d_1969,1,evaluation,FOODS_3_827_WI_3_evaluation


In [26]:
# # --------------------- ADDING CALENDAR AND SELLING PRICES to SALES TRAIN VAL ---------------------- #
df_sales_test = df_sales_test.merge(df_calendar, on = ['d'], how='left')
df_sales_test = df_sales_test.merge(df_selling_prices, on = ["store_id", "item_id", "wm_yr_wk"], how='left')

In [27]:
df_sales_test.head()

,id,item_id,store_id,dept_id,cat_id,state_id,demand,d,set,strategy,original_id,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_tomorrow_1,event_tomorrow_2,event_type_tomorrow_1,event_type_tomorrow_2,sell_price,sell_price_cents,sell_price_preceived
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,CA_1,HOBBIES_1,HOBBIES,CA,0,d_1914,1,validation,HOBBIES_1_001_CA_1_validation,2016-04-25,11613,Monday,3,4,2016,30,4,30,4,0,0,0,30,30,4,4,8.38,0.38,8
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,CA_1,HOBBIES_1,HOBBIES,CA,0,d_1914,1,validation,HOBBIES_1_002_CA_1_validation,2016-04-25,11613,Monday,3,4,2016,30,4,30,4,0,0,0,30,30,4,4,3.97,0.97,3
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,CA_1,HOBBIES_1,HOBBIES,CA,0,d_1914,1,validation,HOBBIES_1_003_CA_1_validation,2016-04-25,11613,Monday,3,4,2016,30,4,30,4,0,0,0,30,30,4,4,2.97,0.97,2
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,CA_1,HOBBIES_1,HOBBIES,CA,0,d_1914,1,validation,HOBBIES_1_004_CA_1_validation,2016-04-25,11613,Monday,3,4,2016,30,4,30,4,0,0,0,30,30,4,4,4.64,0.64,4
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,CA_1,HOBBIES_1,HOBBIES,CA,0,d_1914,1,validation,HOBBIES_1_005_CA_1_validation,2016-04-25,11613,Monday,3,4,2016,30,4,30,4,0,0,0,30,30,4,4,2.88,0.88,2


In [28]:
# ---------------------- Making {id}_encoded features ------------------------ #
for num, col in enumerate(['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id']):
    df_sales_test[f'{col}_encoded'] = le[num].transform(df_sales_test[col])

In [29]:
df_sales_test.columns

Index(['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id', 'demand',
       'd', 'set', 'strategy', 'original_id', 'date', 'wm_yr_wk', 'weekday',
       'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'event_tomorrow_1',
       'event_tomorrow_2', 'event_type_tomorrow_1', 'event_type_tomorrow_2',
       'sell_price', 'sell_price_cents', 'sell_price_preceived', 'id_encoded',
       'item_id_encoded', 'store_id_encoded', 'dept_id_encoded',
       'cat_id_encoded', 'state_id_encoded'],
      dtype='object')

In [30]:
# -------------------------- TRAIN VAL TEST -------------------------- #
df_sales_test = df_sales_test.drop(columns=['original_id'])
df_train_val_test = pd.concat(objs=[df_sales_train_val, df_sales_test], axis=0, ignore_index=True)
df_train_val_test = reduce_mem_usage(df_train_val_test)

Memory usage of dataframe is 4830.71 MB
Memory usage after optimization is: 1768.96 MB
Decreased by 63.4%


In [31]:
df_train_val_test['d_number'] = df_train_val_test['d'].str[2:].astype(int)

In [32]:
df_train_val_test = df_train_val_test.sort_values(by=['id', 'd_number'])

In [33]:
df_train_val_test = df_train_val_test.reset_index(drop=True)

## Feature Engineering

In [5]:
df_train_val_test = simple_fe_extra(df_train_val_test)

NameError: name 'df_train_val_test' is not defined

In [35]:
for col in df_train_val_test.columns:
    if is_numeric_dtype(df_train_val_test[col]):
        print(col)
        df_train_val_test[col] = df_train_val_test[col].round(decimals=3)

demand
set
wm_yr_wk
wday
month
year
event_name_1
event_type_1
event_name_2
event_type_2
snap_CA
snap_TX
snap_WI
event_tomorrow_1
event_tomorrow_2
event_type_tomorrow_1
event_type_tomorrow_2
sell_price
sell_price_cents
sell_price_preceived
id_encoded
item_id_encoded
store_id_encoded
dept_id_encoded
cat_id_encoded
state_id_encoded
d_number
lag_t56
lag_t57
lag_t58
rolling_mean_t7
rolling_std_t7
rolling_mean_t30
rolling_mean_t90
rolling_mean_t180
rolling_std_t30
rolling_skew_t30
rolling_kurt_t30
price_change_t1
price_change_t365
rolling_price_std_t7
rolling_price_std_t30
week
day
dayofweek


In [36]:
df_train_val_test.shape

(30886370, 55)

In [37]:
df_train_val_test.tail()

,id,item_id,store_id,dept_id,cat_id,state_id,d,demand,set,strategy,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_tomorrow_1,event_tomorrow_2,event_type_tomorrow_1,event_type_tomorrow_2,sell_price,sell_price_cents,sell_price_preceived,id_encoded,item_id_encoded,store_id_encoded,dept_id_encoded,cat_id_encoded,state_id_encoded,d_number,lag_t56,lag_t57,lag_t58,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,week,day,dayofweek
30886365,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,d_1965,0,1,evaluation,2016-06-15,11620,Wednesday,5,6,2016,30,4,30,4,0,1,1,30,30,4,4,5.94,0.94,5.0,30489,3048,9,6,2,2,1965,0.0,0.0,0.0,0.0,0.0,0.0,0.044,0.078,0.0,NaN,NaN,0.0,0.0,0.0,0.0,24,15,2
30886366,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,d_1966,0,1,evaluation,2016-06-16,11620,Thursday,6,6,2016,30,4,30,4,0,0,0,30,30,4,4,5.94,0.94,5.0,30489,3048,9,6,2,2,1966,0.0,0.0,0.0,0.0,0.0,0.0,0.044,0.078,0.0,NaN,NaN,0.0,0.0,0.0,0.0,24,16,3
30886367,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,d_1967,0,1,evaluation,2016-06-17,11620,Friday,7,6,2016,30,4,30,4,0,0,0,30,30,4,4,5.94,0.94,5.0,30489,3048,9,6,2,2,1967,0.0,0.0,0.0,0.0,0.0,0.0,0.044,0.078,0.0,NaN,NaN,0.0,0.0,0.0,0.0,24,17,4
30886368,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,d_1968,0,1,evaluation,2016-06-18,11621,Saturday,1,6,2016,30,4,30,4,0,0,0,16,7,3,0,5.94,0.94,5.0,30489,3048,9,6,2,2,1968,0.0,0.0,0.0,0.0,0.0,0.0,0.044,0.078,0.0,NaN,NaN,0.0,0.0,0.0,0.0,24,18,5
30886369,HOUSEHOLD_2_516_WI_3,HOUSEHOLD_2_516,WI_3,HOUSEHOLD_2,HOUSEHOLD,WI,d_1969,0,1,evaluation,2016-06-19,11621,Sunday,2,6,2016,16,3,7,0,0,0,0,30,30,4,4,5.94,0.94,5.0,30489,3048,9,6,2,2,1969,0.0,0.0,0.0,0.0,0.0,0.0,0.044,0.078,0.0,NaN,NaN,0.0,0.0,0.0,0.0,24,19,6


In [38]:
df_train_val_test.to_parquet('train_val_test.pqt', index=False)

## Save time

In [30]:
df_train_val_test = pd.read_parquet('train_val_test.pqt')

In [31]:
df_train_val_test['is_weekend'] = 0
df_train_val_test.loc[df_train_val_test['weekday'].isin(['Saturday', 'Sunday']), 'is_weekend'] = 1 

In [32]:
# df_train_val_test["price_rank_store_item"] = df_train_val_test.groupby(by = ["store_id", "item_id"])['sell_price'].rank(method='dense',ascending=False)

In [33]:
# Making a feature 'prob_high_demand' which is a float which is max on day, day before and day after the event_name_1 
# and which decreases as we move away from event date on both in past and future. Think of it as a bell curve over date
# peaking at event date

indexes = df_train_val_test.groupby(by=['event_name_1']).groups
del indexes[30]                # 30 representes 'nan' event
high_demand_indexes = []
mid_demand_indexes = []
mid_low_demand_indexes = []
for key, val in indexes.items():
    for value in indexes[key]:
        high_demand_indexes.append(value)
        high_demand_indexes.append(value + 1)
        mid_demand_indexes.append(value + 2)
        mid_low_demand_indexes.append(value + 3)
        high_demand_indexes.append(value - 1)
        mid_demand_indexes.append(value - 2)
        mid_low_demand_indexes.append(value - 3)
        
high_demand_indexes = list(set(high_demand_indexes))
mid_demand_indexes = list(set(mid_demand_indexes))
mid_low_demand_indexes = list(set(mid_low_demand_indexes))

df_train_val_test['prob_high_demand'] = 0
high_demand_indexes = list(set(df_train_val_test.index).intersection(set(high_demand_indexes)))
mid_demand_indexes = list(set(df_train_val_test.index).intersection(set(mid_demand_indexes)))
mid_low_demand_indexes = list(set(df_train_val_test.index).intersection(set(mid_low_demand_indexes)))

df_train_val_test.loc[mid_low_demand_indexes, 'prob_high_demand'] = 0.3
df_train_val_test.loc[mid_demand_indexes, 'prob_high_demand'] = 0.6
df_train_val_test.loc[high_demand_indexes, 'prob_high_demand'] = 0.9

In [34]:
df_train_val_test.columns

Index(['id', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id', 'd',
       'demand', 'set', 'strategy', 'date', 'wm_yr_wk', 'weekday', 'wday',
       'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'event_tomorrow_1',
       'event_tomorrow_2', 'event_type_tomorrow_1', 'event_type_tomorrow_2',
       'sell_price', 'sell_price_cents', 'sell_price_preceived', 'id_encoded',
       'item_id_encoded', 'store_id_encoded', 'dept_id_encoded',
       'cat_id_encoded', 'state_id_encoded', 'd_number', 'lag_t56', 'lag_t57',
       'lag_t58', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30',
       'rolling_mean_t90', 'rolling_mean_t180', 'rolling_std_t30',
       'rolling_skew_t30', 'rolling_kurt_t30', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30',
       'week', 'day', 'dayofweek', 'is_weekend', 'prob_high_demand'],
      dtype='object')

In [17]:
df_train_val_test['is_weekend'] = df_train_val_test['is_weekend'].astype('category')

In [18]:
# # tomorrow_some_event is causing over fitting 
# df_train_val_test = df_train_val_test.drop(columns=['event'])

In [19]:
# df_train_val_test.columns

## Modelling

## Cross validation

In [7]:
import lightgbm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error as mse
import time

In [8]:
# df_train_val_test['diff_high_low_demand'] = np.nan
# high_low = prep.HighLow()
# high_low.train(df_train_val_test.loc[df_train_val_test['strategy']=='train', :])
# df_train_val_test = high_low.apply(df_train_val_test)

In [9]:
# df_train_val_test['diff_high_low_demand'].isna().sum()

In [10]:
df_train_val_test.head()

,id,item_id,store_id,dept_id,cat_id,state_id,d,demand,set,strategy,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_tomorrow_1,event_tomorrow_2,event_type_tomorrow_1,event_type_tomorrow_2,sell_price,sell_price_cents,sell_price_preceived,id_encoded,item_id_encoded,store_id_encoded,dept_id_encoded,cat_id_encoded,state_id_encoded,d_number,lag_t56,lag_t57,lag_t58,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,week,day,dayofweek
0,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_957,0,0,train,2013-09-11,11333,Wednesday,5,9,2013,30,4,30,4,0,1,1,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,11,2
1,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_958,0,0,train,2013-09-12,11333,Thursday,6,9,2013,30,4,30,4,0,1,1,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,37,12,3
2,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_959,2,0,train,2013-09-13,11333,Friday,7,9,2013,30,4,30,4,0,1,0,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,37,13,4
3,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_960,1,0,train,2013-09-14,11334,Saturday,1,9,2013,30,4,30,4,0,0,1,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,37,14,5
4,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_961,2,0,train,2013-09-15,11334,Sunday,2,9,2013,30,4,30,4,0,1,1,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,37,15,6


In [11]:
cols = list(df_train_val_test.columns)

In [12]:
cols

['id',
 'item_id',
 'store_id',
 'dept_id',
 'cat_id',
 'state_id',
 'd',
 'demand',
 'set',
 'strategy',
 'date',
 'wm_yr_wk',
 'weekday',
 'wday',
 'month',
 'year',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'event_tomorrow_1',
 'event_tomorrow_2',
 'event_type_tomorrow_1',
 'event_type_tomorrow_2',
 'sell_price',
 'sell_price_cents',
 'sell_price_preceived',
 'id_encoded',
 'item_id_encoded',
 'store_id_encoded',
 'dept_id_encoded',
 'cat_id_encoded',
 'state_id_encoded',
 'd_number',
 'lag_t56',
 'lag_t57',
 'lag_t58',
 'rolling_mean_t7',
 'rolling_std_t7',
 'rolling_mean_t30',
 'rolling_mean_t90',
 'rolling_mean_t180',
 'rolling_std_t30',
 'rolling_skew_t30',
 'rolling_kurt_t30',
 'price_change_t1',
 'price_change_t365',
 'rolling_price_std_t7',
 'rolling_price_std_t30',
 'week',
 'day',
 'dayofweek']

In [13]:
for col in ['event_tomorrow_1', 
            'event_tomorrow_2','event_type_tomorrow_1','event_type_tomorrow_2']:
    cols.remove(col)

In [14]:
X_train = df_train_val_test.loc[df_train_val_test['set']==0, cols]
y_train = X_train['demand']
X_train = X_train.drop(columns=['d_number', 'set', 'demand', 'strategy', 'week', 'day', 'dayofweek',
                                'wday','month']) # drop 'set' bcoz it can cause data leak 

In [15]:
scores = []
m5split2 = M5Split2(n_splits=0, group_id='id', date_col='date', train_size=365, gap_size=0, val_size=28, step=28)
fold = 0
estimators = []
for trn_idx, _, val_idx in m5split2.split(X_train):
    fold_s_time = time.time()
    s_time = time.time()
    X_trn = X_train.loc[trn_idx, :]
    y_trn = y_train[trn_idx]
    X_val = X_train.loc[val_idx, :]
    y_val = y_train[val_idx]
    
    e_time = time.time()
    print(f'Time taken for splitting of fold{fold}: {e_time-s_time} seconds')

    s_time = time.time()
    lgbmr = lightgbm.LGBMRegressor(objective = "poisson",
                                   boosting_type = 'gbdt',
                                   n_estimators = 500,
                                   random_state = 20,
                                   learning_rate = 0.1,
                                   bagging_fraction = 0.75,
                                   bagging_freq = 10, 
                                   colsample_bytree = 0.75)
    lgbmr.fit(X_trn.drop(columns=['date', 'd']), y_trn)
    estimators.append(lgbmr)
    y_predict = np.around(lgbmr.predict(X_val.drop(columns=['date', 'd'])))
    e_time = time.time()
    print(f'Time taken for learning of fold{fold}: {e_time-s_time} seconds')
    
    s_time = time.time()
    rmse_score = np.sqrt(mse(y_val, y_predict))
    print(f'CV RMSE score of fold{fold} (train_len: {len(trn_idx)}, val_len: {len(val_idx)}): {rmse_score}')
    scores.append(rmse_score)
    fold_e_time = time.time()
    print(f'Total time taken for fold{fold}: {fold_e_time-fold_s_time} seconds\n')
    fold += 1

print(f'Mean score: {sum(scores)/len(scores)}, Stdev: {np.std(scores)}')

Time taken for splitting of fold0: 25.687737464904785 seconds
Time taken for learning of fold0: 487.6398332118988 seconds
CV RMSE score of fold0 (train_len: 11982570, val_len: 823230): 2.194780363884547
Total time taken for fold0: 513.3333785533905 seconds

Mean score: 2.194780363884547, Stdev: 0.0


In [16]:
X_train['date'].max()

Timestamp('2016-04-24 00:00:00')

In [17]:
X_trn = X_train.loc[X_train['date']>='2015-04-24 00:00:00']
y_trn = y_train[X_trn.index]
lgbmr = lightgbm.LGBMRegressor(objective = "poisson",
                                   boosting_type = 'gbdt',
                                   n_estimators = 500,
                                   random_state = 20,
                                   learning_rate = 0.1,
                                   bagging_fraction = 0.75,
                                   bagging_freq = 10, 
                                   colsample_bytree = 0.75)
lgbmr.fit(X_trn.drop(columns=['date', 'd']), y_trn)
estimators.append(lgbmr)

## Ensembling

In [18]:
X_test = df_train_val_test.loc[df_train_val_test['set']==1, cols]
# X_test = X_test.drop(columns=['date', 'd_number', 'set', 'demand', 'strategy', 'rolling_5'])
X_test = X_test.drop(columns=['d_number', 'set', 'demand', 'strategy', 'week', 'day', 'dayofweek', 
                              'wday', 'month'])
predictions = pd.DataFrame(columns=[estimator.__class__.__name__ + str(estimators.index(estimator)) for estimator in estimators])

for i, estimator in enumerate(estimators):
    y_predict = estimator.predict(X_test.drop(columns=['date', 'd']))
    predictions[predictions.columns[i]] = pd.Series(y_predict)

cols = list(predictions.columns)
predictions['final'] = np.mean(predictions.loc[:, cols], axis=1)

In [19]:
predictions.max()

LGBMRegressor0    186.187697
LGBMRegressor1    128.709242
final             147.796406
dtype: float64

In [20]:
predictions.min()

LGBMRegressor0    0.002690
LGBMRegressor1    0.003553
final             0.003504
dtype: float64

In [21]:
prediction_col = 'LGBMRegressor0'
predictions.loc[predictions[prediction_col] < 0.0, prediction_col] = 0.0 

predictions = predictions.set_index(X_test.index)

X_test['pred'] = predictions['LGBMRegressor0']


df_train_val_test.loc[X_test.index, 'demand'] = X_test['pred']
df_submit = df_train_val_test.loc[X_test.index, :]
df_submit['original_id'] = df_submit['id'].str.cat(df_submit['strategy'], sep='_')

df_submit.to_csv(f'{INPUT}submit.csv', index=False)

df_submit.head()

,id,item_id,store_id,dept_id,cat_id,state_id,d,demand,set,strategy,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,event_tomorrow_1,event_tomorrow_2,event_type_tomorrow_1,event_type_tomorrow_2,sell_price,sell_price_cents,sell_price_preceived,id_encoded,item_id_encoded,store_id_encoded,dept_id_encoded,cat_id_encoded,state_id_encoded,d_number,lag_t56,lag_t57,lag_t58,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,week,day,dayofweek,original_id
957,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_1914,0.614683,1,validation,2016-04-25,11613,Monday,3,4,2016,30,4,30,4,0,0,0,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,1914,1.0,0.0,0.0,0.286,0.488,0.767,0.689,0.656,0.935,0.774,-0.753,0.0,0.0,0.0,0.0,17,25,0,FOODS_1_001_CA_1_validation
958,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_1915,0.548920,1,validation,2016-04-26,11613,Tuesday,4,4,2016,30,4,30,4,0,0,0,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,1915,0.0,1.0,0.0,0.143,0.378,0.700,0.678,0.656,0.915,0.948,-0.362,0.0,0.0,0.0,0.0,17,26,1,FOODS_1_001_CA_1_validation
959,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_1916,0.561357,1,validation,2016-04-27,11613,Wednesday,5,4,2016,30,4,30,4,0,0,0,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,1916,1.0,0.0,1.0,0.286,0.488,0.733,0.689,0.661,0.907,0.874,-0.425,0.0,0.0,0.0,0.0,17,27,2,FOODS_1_001_CA_1_validation
960,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_1917,0.561955,1,validation,2016-04-28,11613,Thursday,6,4,2016,30,4,30,4,0,0,0,30,30,4,4,2.24,0.24,2.0,0,0,0,0,0,0,1917,0.0,1.0,0.0,0.286,0.488,0.667,0.656,0.661,0.884,1.057,0.057,0.0,0.0,0.0,0.0,17,28,3,FOODS_1_001_CA_1_validation
961,FOODS_1_001_CA_1,FOODS_1_001,CA_1,FOODS_1,FOODS,CA,d_1918,0.615078,1,validation,2016-04-29,11613,Friday,7,4,2016,30,4,30,4,0,0,0,21,30,2,4,2.24,0.24,2.0,0,0,0,0,0,0,1918,0.0,0.0,1.0,0.286,0.488,0.667,0.656,0.661,0.884,1.057,0.057,0.0,0.0,0.0,0.0,17,29,4,FOODS_1_001_CA_1_validation


In [22]:
df_submit = pd.read_csv(f'{INPUT}submit.csv')
df_submit = reduce_mem_usage(df_submit)

Memory usage of dataframe is 729.50 MB
Memory usage after optimization is: 204.84 MB
Decreased by 71.9%


In [23]:
df_submit['d2'] = df_submit['d'].str[2:]
df_submit['d2'] = df_submit['d2'].astype(int)
df_submit.loc[df_submit['strategy']=='validation', 'd2'] = df_submit.loc[df_submit['strategy']=='validation', 'd2'] - 1913
df_submit.loc[df_submit['strategy']=='evaluation', 'd2'] = df_submit.loc[df_submit['strategy']=='evaluation', 'd2'] - 1941
df_submit['d2'] = df_submit['d2'].astype(str)
df_submit['d2'] = 'F' + df_submit['d2']
df_submit.loc[df_submit['strategy']=='evaluation', 'demand'] = 0.0
df_submission = df_submit.loc[:, ['original_id', 'demand', 'd2']].pivot(index='original_id', columns='d2', values='demand')

In [24]:
cols_order = ['F' + str(i) for i in range(1,29)]
df_submission = df_submission[cols_order]

df_sample_submission = pd.read_csv(f'{INPUT}sample_submission.csv')
df_sample_submission = df_sample_submission.set_index(keys='id')
df_submission = df_submission.reindex(df_sample_submission.index)
df_submission = df_submission.reset_index()
df_submission = df_submission.rename(columns={'original_id': 'id'})
df_submission.to_csv(f'{INPUT}submission.csv', index=False)
df_submission

d2,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.897781,0.884908,0.853645,0.885819,0.996278,1.115690,1.125304,0.874587,0.806872,0.818190,0.769559,0.942281,1.110407,0.961774,0.797994,0.757915,0.791706,0.804746,0.976542,1.114257,1.294987,0.994611,0.935994,0.912096,0.895712,1.019846,1.263765,1.224628
1,HOBBIES_1_002_CA_1_validation,0.300033,0.278321,0.262996,0.281288,0.307130,0.388263,0.421058,0.335477,0.313094,0.302366,0.298953,0.339409,0.470204,0.396452,0.334366,0.304620,0.304703,0.295681,0.330479,0.411356,0.414811,0.312736,0.281383,0.287084,0.300449,0.362361,0.467349,0.481501
2,HOBBIES_1_003_CA_1_validation,0.534174,0.478910,0.478133,0.469155,0.537351,0.730551,0.705401,0.598829,0.556227,0.537421,0.512440,0.617800,0.745554,0.666807,0.555953,0.486346,0.502584,0.503953,0.568309,0.707520,0.679573,0.504783,0.455584,0.451544,0.454739,0.537723,0.698036,0.657764
3,HOBBIES_1_004_CA_1_validation,1.617818,1.420238,1.403945,1.402875,1.495124,1.983605,2.175523,1.595056,1.449002,1.316987,1.284630,1.515490,1.943839,1.734320,1.531848,1.228980,1.300744,1.390956,1.726466,2.094668,2.076798,1.633387,1.404397,1.510294,1.652218,1.622717,2.359910,2.457711
4,HOBBIES_1_005_CA_1_validation,1.096467,1.044566,1.059756,1.002206,1.055892,1.389636,1.376305,1.081801,0.995728,0.974956,0.943649,1.202558,1.480087,1.314277,1.106457,0.964894,0.968829,0.959768,1.118241,1.359109,1.418625,1.127540,1.094230,1.070272,1.069960,1.125311,1.353286,1.322058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
df_sub_doe_pator = pd.read_csv('submission_poe_dator_0.49.csv')

In [26]:
df_sub_doe_pator

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.748174,0.648382,0.616866,0.643540,0.881925,0.876391,1.051534,0.750577,0.768482,0.732278,0.892420,0.940190,1.088354,1.031853,0.793422,0.753668,0.777405,0.790092,0.927890,1.212761,1.074997,0.792178,0.714851,0.698843,0.703834,0.879317,1.121686,1.005841
1,FOODS_1_001_CA_2_validation,0.862519,0.906855,0.879643,0.973476,0.967872,1.044653,1.205497,0.903296,0.953694,0.925754,1.032613,1.126745,1.383555,1.266457,1.031187,0.988939,1.027437,1.016480,1.219878,1.468869,1.405831,0.895190,0.873617,0.858570,0.925989,1.116315,1.420866,1.273016
2,FOODS_1_001_CA_3_validation,0.955000,0.828393,0.773412,0.746366,0.860524,1.130992,1.338934,0.998796,0.980547,0.914648,0.969852,0.997275,1.551182,1.408174,1.095415,0.929464,0.904415,0.885724,1.006383,1.480922,1.528007,1.052925,0.904174,0.827126,0.788875,0.938962,1.286009,1.376725
3,FOODS_1_001_CA_4_validation,0.398399,0.347684,0.343555,0.347587,0.419595,0.344572,0.362035,0.419844,0.407615,0.397662,0.345649,0.430763,0.451206,0.387208,0.390565,0.361079,0.402989,0.396133,0.428906,0.418390,0.380917,0.373088,0.358649,0.319483,0.320422,0.355013,0.390877,0.352919
4,FOODS_1_001_TX_1_validation,0.289736,0.293023,0.291305,0.288159,0.416391,0.248531,0.260920,0.911467,0.935519,0.862451,0.770982,0.721093,0.708860,0.607165,0.548070,0.510901,0.502318,0.433236,0.495555,0.458131,0.470257,0.359559,0.394135,0.390145,0.434212,0.463324,0.536218,0.543345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_TX_2_evaluation,0.214382,0.193294,0.218409,0.206152,0.311280,0.312019,0.318882,0.258842,0.236262,0.229228,0.493023,0.441262,0.461439,0.391726,0.272737,0.273925,0.276284,0.272475,0.342058,0.421860,0.344391,0.244175,0.236903,0.235210,0.287532,0.344500,0.443315,0.430130
60976,HOUSEHOLD_2_516_TX_3_evaluation,0.140529,0.125286,0.130615,0.128350,0.181651,0.176063,0.176992,0.105888,0.112739,0.111982,0.203784,0.172199,0.214542,0.189634,0.129458,0.127883,0.129680,0.135359,0.190719,0.259023,0.242682,0.163221,0.156682,0.166008,0.177783,0.247325,0.280407,0.267154
60977,HOUSEHOLD_2_516_WI_1_evaluation,0.053739,0.052284,0.051593,0.058389,0.089141,0.093650,0.088636,0.072232,0.073015,0.068701,0.145973,0.119885,0.126227,0.119807,0.071627,0.068556,0.070554,0.075176,0.116616,0.123143,0.116972,0.069461,0.071798,0.073942,0.078821,0.129165,0.137323,0.128882
60978,HOUSEHOLD_2_516_WI_2_evaluation,0.028988,0.027910,0.029603,0.046807,0.068672,0.059103,0.068980,0.050944,0.053161,0.047963,0.131641,0.116768,0.091098,0.093276,0.064236,0.060988,0.065985,0.067824,0.106160,0.087262,0.083996,0.060191,0.060692,0.061488,0.054804,0.081718,0.079276,0.072634


In [27]:
df_sales_train_eval = pd.read_csv('sales_train_evaluation.csv')

In [28]:
df_sales_train_eval

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,d_300,d_301,d_302,d_303,d_304,d_305,d_306,d_307,d_308,d_309,d_310,d_311,d_312,d_313,d_314,d_315,d_316,d_317,d_318,d_319,d_320,d_321,d_322,d_323,d_324,d_325,d_326,d_327,d_328,d_329,d_330,d_331,d_332,d_333,d_334,d_335,d_336,d_337,d_338,d_339,d_340,d_341,d_342,d_343,d_344,d_345,d_346,d_347,d_348,d_349,d_350,d_351,d_352,d_353,d_354,d_355,d_356,d_357,d_358,d_359,d_360,d_361,d_362,d_363,d_364,d_365,d_366,d_367,d_368,d_369,d_370,d_371,d_372,d_373,d_374,d_375,d_376,d_377,d_378,d_379,d_380,d_381,d_382,d_383,d_384,d_385,d_386,d_387,d_388,d_389,d_390,d_391,d_392,d_393,d_394,d_395,d_396,d_397,d_398,d_399,d_400,d_401,d_402,d_403,d_404,d_405,d_406,d_407,d_408,d_409,d_410,d_411,d_412,d_413,d_414,d_415,d_416,d_417,d_418,d_419,d_420,d_421,d_422,d_423,d_424,d_425,d_426,d_427,d_428,d_429,d_430,d_431,d_432,d_433,d_434,d_435,d_436,d_437,d_438,d_439,d_440,d_441,d_442,d_443,d_444,d_445,d_446,d_447,d_448,d_449,d_450,d_451,d_452,d_453,d_454,d_455,d_456,d_457,d_458,d_459,d_460,d_461,d_462,d_463,d_464,d_465,d_466,d_467,d_468,d_469,d_470,d_471,d_472,d_473,d_474,d_475,d_476,d_477,d_478,d_479,d_480,d_481,d_482,d_483,d_484,d_485,d_486,d_487,d_488,d_489,d_490,d_491,d_492,d_493,...,d_1443,d_1444,d_1445,d_1446,d_1447,d_1448,d_1449,d_1450,d_1451,d_1452,d_1453,d_1454,d_1455,d_1456,d_1457,d_1458,d_1459,d_1460,d_1461,d_1462,d_1463,d_1464,d_1465,d_1466,d_1467,d_1468,d_1469,d_1470,d_1471,d_1472,d_1473,d_1474,d_1475,d_1476,d_1477,d_1478,d_1479,d_1480,d_1481,d_1482,d_1483,d_1484,d_1485,d_1486,d_1487,d_1488,d_1489,d_1490,d_1491,d_1492,d_1493,d_1494,d_1495,d_1496,d_1497,d_1498,d_1499,d_1500,d_1501,d_1502,d_1503,d_1504,d_1505,d_1506,d_1507,d_1508,d_1509,d_1510,d_1511,d_1512,d_1513,d_1514,d_1515,d_1516,d_1517,d_1518,d_1519,d_1520,d_1521,d_1522,d_1523,d_1524,d_1525,d_1526,d_1527,d_1528,d_1529,d_1530,d_1531,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,d_1552,d_1553,d_1554,d_1555,d_1556,d_1557,d_1558,d_1559,d_1560,d_1561,d_1562,d_1563,d_1564,d_1565,d_1566,d_1567,d_1568,d_1569,d_1570,d_1571,d_1572,d_1573,d_1574,d_1575,d_1576,d_1577,d_1578,d_1579,d_1580,d_1581,d_1582,d_1583,d_1584,d_1585,d_1586,d_1587,d_1588,d_1589,d_1590,d_1591,d_1592,d_1593,d_1594,d_1595,d_1596,d_1597,d_1598,d_1599,d_